In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time
import pickle
from dateutil import relativedelta
import seaborn as sns

### API call for redcap

In [2]:
def api_call(url, query, logger=None):
    """ helper function to make API calls to RedCap
    """
    r = requests.post(url, data=query, verify=False)
    http_status = str(r.status_code)
    print(f'HTTP Status: {http_status}')

    if http_status == "200":
        query_results = r.json()
        query_df = pd.DataFrame(query_results)

    else:
        print(f"RedCap API request Failed with HTTP Status: {http_status}")
        query_df = None
        
    return query_df

def get_inventory_count(df, index_col, availability_indicators):
    """ helper function to count participants with recorded data in redcap
    """
    assess_cols = df.columns.drop(index_col)

    if availability_indicators == 'number':
        df = df.replace("", np.nan)
        df[assess_cols] = df[assess_cols].astype(np.float64)

    inventory = {}
    for col in assess_cols:        
        if availability_indicators == 'number':
            availability_count = df[~df[col].isna()][index_col].nunique()
        else:
            availability_count = df[df[col].isin(availability_indicators)][index_col].nunique()
        inventory[col] = availability_count
    return inventory

def get_available_data(config_json, DATASET_ROOT, var_name, preferred_var_source="primary"):
    """ Get data for given variables from available sources
        All return dataframes should have participant_id and visit_id as index
    """
    config_data = json.load(open(config_json))
    data_sources = config_data['data_sources']
    variable_info = config_data['variables'][var_name]
    variable_type = variable_info["type"]
    variable_sources = variable_info["sources"]

    if preferred_var_source == "primary":
        selected_var_source = variable_info['primary_source']
        selected_var_instrument = variable_info['primary_instrument']
    elif preferred_var_source == "secondary":
        selected_var_source = variable_info['secondary_source']
        selected_var_instrument = variable_info['secondary_instrument']
    else:
        print(f"Using preferred source {preferred_var_source} for variable {var_name}")
        preferred_var_data_source = preferred_var_source["data_source"]
        preferred_var_instrument = preferred_var_source["instrument"]

        if preferred_var_data_source not in variable_sources.keys():
            print(f"Preferred data source {preferred_var_data_source} not available for variable {var_name}")
            return None
        else:
            selected_var_source = preferred_var_data_source

        if preferred_var_instrument not in variable_sources[selected_var_source].keys():
            print(f"Preferred var instrument {preferred_var_instrument} not available for variable {var_name}")
            return None
        else:
            selected_var_instrument = preferred_var_instrument

    print(f"Using variable {var_name} from source {selected_var_source} and instrument {selected_var_instrument}")

    external_var_cols = variable_sources[selected_var_source][selected_var_instrument]

    # Get data from primary source
    var_file = data_sources[selected_var_source][selected_var_instrument]["path"]
    var_file_path = f"{DATASET_ROOT}/{var_file}"
    var_file_index = data_sources[selected_var_source][selected_var_instrument]["index_cols"]

    var_df = pd.read_csv(var_file_path)
    selected_var_cols = list(set(var_file_index + external_var_cols))
    var_df = var_df[selected_var_cols]
    
    if (variable_type == "date") & (len(external_var_cols) == 1):
        var_df[external_var_cols[0]] = pd.to_datetime(var_df[external_var_cols[0]], errors="coerce", dayfirst=False)

    if (len(external_var_cols) == 1):
        var_df = var_df.rename(columns={external_var_cols[0]:var_name})
    
    return var_df


### Paths


In [3]:
DATASET_ROOT = "/home/nikhil/projects/Parkinsons/qpn/"

# Current nipoppy manifest
release_dir = f"{DATASET_ROOT}/releases/"
current_release = "Jan_2024"

tabular_data_release_dir = f"{release_dir}/{current_release}/"

demo_config_json = "../workflow/tabular/demographics.json"
pheno_config_json = "../workflow/tabular/pheno.json"


### Standardized index names

In [4]:
baseline_event_name = "Baseline (Arm 1: C-OPN)"

## redcap event name variations
config_data = json.load(open(demo_config_json))
data_sources = config_data['data_sources']
redcap_data_sources = data_sources['redcap']

redcap_field_name_map = {}

for instrument in redcap_data_sources.keys():
    index_cols = redcap_data_sources[instrument]['index_cols']
    record_id = index_cols[0]
    event_name = index_cols[1]

    redcap_field_name_map[record_id] = "participant_id"
    redcap_field_name_map[event_name] = "redcap_event_name"
print(f"redcap_field_name_map: {redcap_field_name_map}")

# legacy participant_id variations in DOB and BD_RPQ
legacy_field_name_map = {}
legacy_field_name_map['Record ID'] = "participant_id"
legacy_field_name_map['Patient #'] = "participant_id"
legacy_field_name_map['Name of visit (V01, V02, V03)'] = "visit"
print(f"legacy_field_name_map: {legacy_field_name_map}")

redcap_field_name_map: {'Record ID:': 'participant_id', 'Event Name': 'redcap_event_name', 'record_id': 'participant_id', 'redcap_event_name': 'redcap_event_name'}
legacy_field_name_map: {'Record ID': 'participant_id', 'Patient #': 'participant_id', 'Name of visit (V01, V02, V03)': 'visit'}


### Update RedCAP reports through API 
(Not updating extended report since it has to come from Sarah)
- "global_records_query"
- "QPN MoCA-UPDRS-Neuropsy data_Sarah"

In [5]:
update_redcap_reports = False

redcap_report_list = ["global_records_query", "QPN MoCA-UPDRS-Neuropsy data_Sarah"]
if update_redcap_reports:
    redcap_config_json = f"{DATASET_ROOT}/proc/.redcap.json"
    redcap_config = json.load(open(redcap_config_json))
    url = redcap_config["url"]
    
    for redcap_report in redcap_report_list:
        print(f"Getting data for RedCap report: {redcap_report}")
        records_query = redcap_config["queries"][redcap_report]
        query_df = api_call(url, records_query, logger=None)
        report_csv = f"{release_dir}{current_release}/tabular/redcap/{redcap_report}.csv"
        query_df.to_csv(report_csv, index=False)
        print(f"Saved RedCap report to {report_csv}")



### Available participants

In [6]:
QPN_participants_df = get_available_data(demo_config_json,tabular_data_release_dir,"participant_id")
QPN_participants = QPN_participants_df["participant_id"].unique()
n_participants = len(QPN_participants)
print(f"Number of participants: {n_participants}")

Using variable participant_id from source local and instrument manifest
Number of participants: 303


### Fetch demographic data

In [157]:
demo_vars = ["dob", "group", "sex", "education"]
# preferred_var_source = {"data_source":"local","instrument":"legacy_DOB"}
vars_with_secondary_source = ["dob"]

config_json = demo_config_json
index_cols = ["participant_id", "redcap_event_name"]

demo_var_df = pd.DataFrame()
for var in demo_vars:
    _df = get_available_data(config_json,tabular_data_release_dir,var)
    _df = _df.rename(columns=redcap_field_name_map)
    _df = _df.rename(columns=legacy_field_name_map)
    _df = _df[_df["participant_id"].isin(QPN_participants)].copy()

    if var in vars_with_secondary_source:
        print(f"**Getting data from the secondary source for variable {var}**")
        _df2 = get_available_data(config_json,tabular_data_release_dir,var,preferred_var_source="secondary")
        _df2 = _df2.rename(columns=legacy_field_name_map)
        _df2 = _df2.rename(columns={var:var+"_secondary"})
        _df2 = _df2[_df2["participant_id"].isin(QPN_participants)].copy()
        
        # Merge primary and secondary sources
        n_missing_in_primary = _df[_df["redcap_event_name"]==baseline_event_name][var].isna().sum()
        print(f"Missing data in primary source: {n_missing_in_primary}")

        if "redcap_event_name" in _df2.columns:
            _df = pd.merge(_df, _df2, on=["participant_id","redcap_event_name"], how="outer")
        else:
            _df = pd.merge(_df, _df2, on="participant_id", how="outer")
        _df[var] = _df[var].fillna(_df[var+"_secondary"])
        # _df = _df.drop(columns=[var+"_secondary"])

        n_missing_after_secondary_fill = _df[_df["redcap_event_name"]==baseline_event_name][var].isna().sum()
        print(f"Missing data after secondary source fill: {n_missing_after_secondary_fill}")

    if demo_var_df.empty:
        demo_var_df = _df
    else:
        demo_var_df = pd.merge(demo_var_df, _df, on=index_cols, how="outer")   

demo_participants = demo_var_df["participant_id"].unique()
n_demo_participants = len(demo_participants)
print('-'*50)
print(f"Number of participants with demographics data: {n_demo_participants}")
print('-'*50)

demo_redcap_events = demo_var_df["redcap_event_name"].unique()
print(f"Demographics data available for events: {demo_redcap_events}")
print('-'*50)

for var in demo_vars:
    n_unique = demo_var_df[demo_var_df["redcap_event_name"]=="Baseline (Arm 1: C-OPN)"][var].nunique()
    n_missing = demo_var_df[demo_var_df["redcap_event_name"]=="Baseline (Arm 1: C-OPN)"][var].isna().sum()
    print(f"Var: {var}, n_unique: {n_unique}, n_missing: {n_missing} (out of {n_demo_participants})")

demo_var_df.head()

Using variable dob from source redcap and instrument sarah_extended_export
**Getting data from the secondary source for variable dob**
Using variable dob from source local and instrument legacy_DOB
Missing data in primary source: 12
Missing data after secondary source fill: 1
Using variable group from source redcap and instrument sarah_extended_export
Using variable sex from source redcap and instrument sarah_extended_export
Using variable education from source redcap and instrument sarah_extended_export
--------------------------------------------------
Number of participants with demographics data: 296
--------------------------------------------------
Demographics data available for events: ['Baseline (Arm 1: C-OPN)' '18 Months Follow-Up/Suivi (Arm 1: C-OPN)'
 '12 Months Follow-Up/Suivi (Arm 1: C-OPN)'
 '72 Months Follow-Up/Suivi (Arm 1: C-OPN)'
 '36 Months Follow-Up/Suivi (Arm 1: C-OPN)'
 '54 Months Follow-Up/Suivi (Arm 1: C-OPN)'
 '24 Months Follow-Up/Suivi (Arm 1: C-OPN)' nan]
--

,redcap_event_name,participant_id,dob,dob_secondary,group,sex,education
0,Baseline (Arm 1: C-OPN),MNI0028,1963-07-27,NaT,PD (Parkinson's Disease)/Maladie de Parkinson,Male/Masculin,NaN
1,Baseline (Arm 1: C-OPN),MNI0056,1942-05-21,NaT,PD (Parkinson's Disease)/Maladie de Parkinson,Male/Masculin,NaN
2,Baseline (Arm 1: C-OPN),MNI0058,1964-03-14,NaT,PD (Parkinson's Disease)/Maladie de Parkinson,Male/Masculin,NaN
3,Baseline (Arm 1: C-OPN),MNI0068,1952-05-08,NaT,PD (Parkinson's Disease)/Maladie de Parkinson,Male/Masculin,NaN
4,Baseline (Arm 1: C-OPN),MNI0079,1971-11-25,NaT,PD (Parkinson's Disease)/Maladie de Parkinson,Female/Féminin,NaN


### Find records with phenotypic data

In [8]:
pheno_vars = ["diagnosis", "updrs_scores", "moca_scores", "diagnosis_date", "updrs_date", "moca_date"]
# preferred_var_source = {"data_source":"local","instrument":"legacy_DOB"}

config_json = pheno_config_json
index_cols = ["participant_id", "redcap_event_name"]
pheno_var_df = pd.DataFrame()
for var in pheno_vars:
    _df = get_available_data(config_json,tabular_data_release_dir,var)
    _df = _df.rename(columns=redcap_field_name_map)
    _df = _df.rename(columns=legacy_field_name_map)
    _df = _df[_df["participant_id"].isin(QPN_participants)].copy()
    if pheno_var_df.empty:
        pheno_var_df = _df
    else:
        pheno_var_df = pd.merge(pheno_var_df, _df, on=index_cols, how="outer")   

pheno_participants = pheno_var_df["participant_id"].unique()
n_pheno_participants = len(pheno_participants)
print('-'*50)
print(f"Number of participants with pheno data: {n_pheno_participants}")
print('-'*50)

pheno_redcap_events = pheno_var_df["redcap_event_name"].unique()
print(f"Pheno data available for events: {pheno_redcap_events}")
print('-'*50)

for var in pheno_var_df.columns:
    for redcap_event in pheno_redcap_events:
        if var not in index_cols:
            pheno_var_event_df = pheno_var_df[pheno_var_df["redcap_event_name"]==redcap_event].copy()
            n_pheno_var_event_participants = pheno_var_event_df["participant_id"].nunique()
            if pheno_var_event_df[var].nunique() > 0:    
                print(f"Var: {var}, Event: {redcap_event}")
                n_unique = pheno_var_event_df[var].nunique()
                n_missing = pheno_var_event_df[var].isna().sum()
                print(f"n_unique: {n_unique}, n_missing: {n_missing} (out of {n_pheno_var_event_participants})")
    print('-'*50)

pheno_var_df.head()

Using variable diagnosis from source redcap and instrument sarah_extended_export
Using variable updrs_scores from source redcap and instrument sarah_extended_export
Using variable moca_scores from source redcap and instrument sarah_extended_export
Using variable diagnosis_date from source redcap and instrument sarah_extended_export
Using variable updrs_date from source redcap and instrument sarah_extended_export


Using variable moca_date from source redcap and instrument sarah_extended_export
--------------------------------------------------
Number of participants with pheno data: 291
--------------------------------------------------
Pheno data available for events: ['Baseline (Arm 1: C-OPN)' '18 Months Follow-Up/Suivi (Arm 1: C-OPN)'
 '12 Months Follow-Up/Suivi (Arm 1: C-OPN)'
 '72 Months Follow-Up/Suivi (Arm 1: C-OPN)'
 '36 Months Follow-Up/Suivi (Arm 1: C-OPN)'
 '54 Months Follow-Up/Suivi (Arm 1: C-OPN)'
 '24 Months Follow-Up/Suivi (Arm 1: C-OPN)']
--------------------------------------------------
Var: diagnosis, Event: Baseline (Arm 1: C-OPN)
n_unique: 5, n_missing: 117 (out of 291)
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
Var: Part III: Motor Examination, Event: Baseline (Arm 1: C-OPN)
n_unique: 53, n_missing: 135 (out of 291)
Var: Part III: Motor Examination, Event: 18 Months

,diagnosis,redcap_event_name,participant_id,Part III: Motor Examination,Part IV: Motor Complications,Part I: Non-Motor Aspects of Experiences of Daily Living (nM-EDL),Part II: Motor Aspects of Experiences of Daily Living (M-EDL),Hoehn and Yahr Stage:,moca_scores,diagnosis_date,updrs_date,moca_date
0,0.0,Baseline (Arm 1: C-OPN),MNI0028,29.0,0.0,0.0,2.0,(2) Bilateral involvement without impairment o...,28.0,2020-09-13,2023-10-04,2023-10-04
1,NaN,Baseline (Arm 1: C-OPN),MNI0056,58.0,0.0,12.0,22.0,(3) Bilateral disease: mild to moderate disabi...,22.0,2017-09-01,2021-06-11,2021-06-11
2,0.0,Baseline (Arm 1: C-OPN),MNI0058,26.0,0.0,0.0,6.0,"(1) Unilateral involvement only, usually with ...",25.0,2020-05-22,2021-07-23,2021-07-23
3,0.0,Baseline (Arm 1: C-OPN),MNI0068,28.0,0.0,11.0,8.0,(2) Bilateral involvement without impairment o...,NaN,2014-01-01,2021-08-27,NaT
4,0.0,Baseline (Arm 1: C-OPN),MNI0079,22.0,8.0,13.0,10.0,(2) Bilateral involvement without impairment o...,26.0,2017-05-01,2022-01-21,2021-12-22


### Add mri_acq date
- Needs to map to redcap_event_name

In [9]:
var = "MRI_date"
config_json = pheno_config_json
mri_date_df = get_available_data(config_json,tabular_data_release_dir,var)
mri_date_df["MRI_date"] = pd.to_datetime(mri_date_df["MRI_date"], errors="coerce", dayfirst=False)

n_mri_participants = mri_date_df["participant_id"].nunique()
print(f"Number of participants with MRI data: {n_mri_participants}")

n_sessions = mri_date_df["session"].nunique()
print(f"Number of MRI sessions: {n_sessions}")

participants_with_follow_ups = mri_date_df[mri_date_df["participant_id"].duplicated()]["participant_id"].unique()
n_participants_with_follow_ups = len(participants_with_follow_ups)
print(f"Number of participants with follow-up MRI: {n_participants_with_follow_ups}")

mri_ses01_date_df = mri_date_df[mri_date_df["session"]=="ses-01"].copy()
mri_ses01_date_df["redcap_event_name"] = "Baseline (Arm 1: C-OPN)"

mri_ses02_date_df = mri_date_df[mri_date_df["session"]=="ses-02"].copy()
mri_ses02_participants = mri_ses02_date_df["participant_id"].unique()
print(f"Number of participants with ses-02 MRI: {len(mri_ses02_participants)}")

baseline_df = mri_ses01_date_df[mri_ses01_date_df["participant_id"].isin(mri_ses02_participants)].set_index("participant_id")
followup_df = mri_ses02_date_df.set_index("participant_id")

visit_months = [12, 18, 24, 30, 36, 42, 48, 54]
month_bins = [9, 15, 21, 27, 33, 39, 45, 51, 57]

event_str_suffix = "Months Follow-Up/Suivi (Arm 1: C-OPN)"
event_names = [f"{m} {event_str_suffix}" for m in visit_months]

# --- Bin the months --- #
followup_df["months_since_baseline"] = followup_df["MRI_date"].dt.to_period('M').astype(int) - baseline_df["MRI_date"].dt.to_period('M').astype(int)
followup_df["months_since_baseline"] = followup_df["months_since_baseline"].replace({0:np.nan}) # Some visits get same acq_date from brodacasting merge. 

followup_df["redcap_event_name"] = pd.cut(followup_df["months_since_baseline"], bins=month_bins, labels=event_names)

mri_date_redcap_event_df = pd.concat([mri_ses01_date_df, followup_df.reset_index()], axis=0)
# mri_date_redcap_event_df = mri_date_redcap_event_df

mri_date_redcap_event_df.sort_values(["participant_id","session"]).head()

Using variable MRI_date from source local and instrument MRI_dates
Number of participants with MRI data: 298
Number of MRI sessions: 2
Number of participants with follow-up MRI: 51
Number of participants with ses-02 MRI: 51


,session,participant_id,MRI_date,redcap_event_name,months_since_baseline
0,ses-01,MNI0028,2023-10-04,Baseline (Arm 1: C-OPN),NaN
1,ses-01,MNI0056,2021-08-18,Baseline (Arm 1: C-OPN),NaN
2,ses-01,MNI0058,2021-08-18,Baseline (Arm 1: C-OPN),NaN
3,ses-01,MNI0068,2021-08-27,Baseline (Arm 1: C-OPN),NaN
4,ses-01,MNI0079,2021-12-22,Baseline (Arm 1: C-OPN),NaN


#### Add MRI date to pheno data

In [10]:
pheno_var_df = pd.merge(pheno_var_df, mri_date_redcap_event_df, on=index_cols, how="right")  
var = "MRI_date"
for redcap_event in mri_date_redcap_event_df["redcap_event_name"].unique():    
    pheno_var_event_df = pheno_var_df[pheno_var_df["redcap_event_name"]==redcap_event].copy()
    n_pheno_var_event_participants = pheno_var_event_df["participant_id"].nunique()
    if pheno_var_event_df[var].nunique() > 0:    
        print(f"Var: {var}, Event: {redcap_event}")
        n_unique = pheno_var_event_df[var].nunique()
        n_missing = pheno_var_event_df[var].isna().sum()
        print(f"n_unique: {n_unique}, n_missing: {n_missing} (out of {n_pheno_var_event_participants})")
pheno_var_df.head()

Var: MRI_date, Event: Baseline (Arm 1: C-OPN)
n_unique: 234, n_missing: 0 (out of 298)
Var: MRI_date, Event: 12 Months Follow-Up/Suivi (Arm 1: C-OPN)
n_unique: 27, n_missing: 0 (out of 30)
Var: MRI_date, Event: 18 Months Follow-Up/Suivi (Arm 1: C-OPN)
n_unique: 8, n_missing: 0 (out of 9)
Var: MRI_date, Event: 48 Months Follow-Up/Suivi (Arm 1: C-OPN)
n_unique: 1, n_missing: 0 (out of 1)
Var: MRI_date, Event: 42 Months Follow-Up/Suivi (Arm 1: C-OPN)
n_unique: 2, n_missing: 0 (out of 2)
Var: MRI_date, Event: 36 Months Follow-Up/Suivi (Arm 1: C-OPN)
n_unique: 1, n_missing: 0 (out of 1)
Var: MRI_date, Event: 30 Months Follow-Up/Suivi (Arm 1: C-OPN)
n_unique: 1, n_missing: 0 (out of 1)


,diagnosis,redcap_event_name,participant_id,Part III: Motor Examination,Part IV: Motor Complications,Part I: Non-Motor Aspects of Experiences of Daily Living (nM-EDL),Part II: Motor Aspects of Experiences of Daily Living (M-EDL),Hoehn and Yahr Stage:,moca_scores,diagnosis_date,updrs_date,moca_date,session,MRI_date,months_since_baseline
0,0.0,Baseline (Arm 1: C-OPN),MNI0028,29.0,0.0,0.0,2.0,(2) Bilateral involvement without impairment o...,28.0,2020-09-13,2023-10-04,2023-10-04,ses-01,2023-10-04,NaN
1,NaN,Baseline (Arm 1: C-OPN),MNI0056,58.0,0.0,12.0,22.0,(3) Bilateral disease: mild to moderate disabi...,22.0,2017-09-01,2021-06-11,2021-06-11,ses-01,2021-08-18,NaN
2,0.0,Baseline (Arm 1: C-OPN),MNI0058,26.0,0.0,0.0,6.0,"(1) Unilateral involvement only, usually with ...",25.0,2020-05-22,2021-07-23,2021-07-23,ses-01,2021-08-18,NaN
3,0.0,Baseline (Arm 1: C-OPN),MNI0068,28.0,0.0,11.0,8.0,(2) Bilateral involvement without impairment o...,NaN,2014-01-01,2021-08-27,NaT,ses-01,2021-08-27,NaN
4,0.0,Baseline (Arm 1: C-OPN),MNI0079,22.0,8.0,13.0,10.0,(2) Bilateral involvement without impairment o...,26.0,2017-05-01,2022-01-21,2021-12-22,ses-01,2021-12-22,NaN


### Neuropsych data
- Comes from either from Sarah's extended report or BD_RPQ_UPDATE_Neuropsy

In [11]:
neuropsych_vars = ["neuropsy_scores","neuropsy_date"]

index_cols = ["participant_id", "visit", "TimePoint (based on REDCap; baseline, 18m, 36m, etc.)", "Délai depuis baseline (mois)"]
neuropsych_df = pd.DataFrame()
for var in neuropsych_vars:
    _df = get_available_data(config_json,tabular_data_release_dir,var)
    _df = _df.rename(columns=redcap_field_name_map)
    _df = _df.rename(columns=legacy_field_name_map)
    _df = _df[_df["participant_id"].isin(QPN_participants)].copy()
    if neuropsych_df.empty:
        neuropsych_df = _df
    else:
        neuropsych_df = pd.merge(neuropsych_df, _df, on=index_cols, how="left")   

neuropsych_participants = neuropsych_df["participant_id"].unique()
n_neuropsych_participants = len(neuropsych_participants)
print('-'*50)
print(f"Number of participants with neuropysch data: {n_neuropsych_participants}")
print('-'*50)

neuropsych_visits = neuropsych_df["visit"].unique()
print(f"neuropsych data available for events: {neuropsych_visits}")
print('-'*50)

neuropsych_df.head()

Using variable neuropsy_scores from source local and instrument BD_RPQ_UPDATE_Neuropsy
Using variable neuropsy_date from source local and instrument BD_RPQ_UPDATE_Neuropsy
--------------------------------------------------
Number of participants with neuropysch data: 167
--------------------------------------------------
neuropsych data available for events: ['V01' 'V02' 'V03']
--------------------------------------------------


/tmp/ipykernel_321430/4091902172.py:83: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  var_df[external_var_cols[0]] = pd.to_datetime(var_df[external_var_cols[0]], errors="coerce", dayfirst=False)


,Stroop GOLDEN: INK Number of Responses,Letter Fluency Total (Repetitions errors),Letter Fluency A (Raw score),"Stroop - D-Kefs, Cond. 1 - COLORS : Time (sec) (Raw score)",Stroop - D-Kefs - Cond. 3 - 1 (time) [automatique],RCFT Copy (Raw score),Semantic fluency TOTAL [automatique],Digit Span Backward - total correct (Raw score),Stroop - D-Kefs - Cond. 3 - 1 (time),Sementic Fluency Total (Repetitions errors),...,Stroop - D-Kefs - Cond.3: Total errors (Raw score),Stroop - D-Kefs - Cond.2 WORDS: Time (sec) (Raw score),Letter fluency TOTAL [automatique],Digit span backward - longest correct serie (Raw score),Semantic Fluency Actions (Raw score),BNT avec IP,"STROOP GOLDEN : words, self-corrected errors (raw score)",TMT B errors (Raw score),"STROOP GOLDEN : colors, self-corrected errors (raw scores)",neuropsy_date
0,999.0,NaN,18.0,NaN,0.0,33,50.0,999.0,NaN,994.0,...,NaN,NaN,55,999.0,25.0,999.0,999.0,999.0,999.0,2017-12-07
1,66.0,994.0,10.0,997.0,999.0,27,34.0,997.0,997.0,994.0,...,997.0,997.0,35,997.0,17.0,994.0,1.0,0.0,1.0,2018-03-12
2,26.0,994.0,6.0,997.0,999.0,"18,5",36.0,997.0,997.0,994.0,...,997.0,997.0,22,997.0,15.0,994.0,0.0,0.0,0.0,2018-03-12
3,999.0,NaN,13.0,NaN,0.0,999,35.0,999.0,NaN,NaN,...,NaN,NaN,32,999.0,18.0,999.0,999.0,999.0,999.0,2018-08-06
4,999.0,994.0,20.0,997.0,999.0,34,44.0,997.0,997.0,994.0,...,997.0,997.0,54,997.0,22.0,53.0,999.0,0.0,999.0,2018-07-31


### Basic clean-up and data checks

In [12]:
# Fix dtypes
for series_name, series in neuropsych_df.items():
    if "score" in series_name:
        if series.dtype == 'object':
            print(f"recasting {series_name} to float by replacing , with .")
            neuropsych_df[series_name] = neuropsych_df[series_name].str.replace(",",".").astype(float)
            neuropsych_df.loc[neuropsych_df[series_name]>900, series_name] = np.nan
            
    # Replace >900 with NaNs
    if series.dtype == 'float':
        neuropsych_df.loc[neuropsych_df[series_name]>900, series_name] = np.nan

# assign redcap_event_name
visit_months = [12, 18, 24, 30, 36, 42, 48, 54]
month_bins = [9, 15, 21, 27, 33, 39, 45, 51, 57]
event_str_suffix = "Months Follow-Up/Suivi (Arm 1: C-OPN)"
event_names = [f"{m} {event_str_suffix}" for m in visit_months]

neuropsych_df["redcap_event_name"] = pd.cut(neuropsych_df["Délai depuis baseline (mois)"], bins=month_bins, labels=event_names).astype(str)
neuropsych_df.loc[neuropsych_df["TimePoint (based on REDCap; baseline, 18m, 36m, etc.)"]=="baseline", 
                  "redcap_event_name"] = "Baseline (Arm 1: C-OPN)"

# Merge with pheno_var_df
index_cols = ["participant_id", "redcap_event_name"]
pheno_var_df = pd.merge(pheno_var_df, neuropsych_df, on=index_cols, how="left")  

pheno_var_df.head()

recasting RCFT Copy (Raw score) to float by replacing , with .
recasting RCFT Delayed (Raw score) to float by replacing , with .
recasting RCFT Immediate (Raw score) to float by replacing , with .


,diagnosis,redcap_event_name,participant_id,Part III: Motor Examination,Part IV: Motor Complications,Part I: Non-Motor Aspects of Experiences of Daily Living (nM-EDL),Part II: Motor Aspects of Experiences of Daily Living (M-EDL),Hoehn and Yahr Stage:,moca_scores,diagnosis_date,...,Stroop - D-Kefs - Cond.3: Total errors (Raw score),Stroop - D-Kefs - Cond.2 WORDS: Time (sec) (Raw score),Letter fluency TOTAL [automatique],Digit span backward - longest correct serie (Raw score),Semantic Fluency Actions (Raw score),BNT avec IP,"STROOP GOLDEN : words, self-corrected errors (raw score)",TMT B errors (Raw score),"STROOP GOLDEN : colors, self-corrected errors (raw scores)",neuropsy_date
0,0.0,Baseline (Arm 1: C-OPN),MNI0028,29.0,0.0,0.0,2.0,(2) Bilateral involvement without impairment o...,28.0,2020-09-13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,NaN,Baseline (Arm 1: C-OPN),MNI0056,58.0,0.0,12.0,22.0,(3) Bilateral disease: mild to moderate disabi...,22.0,2017-09-01,...,4.0,28.0,17.0,3.0,8.0,56.0,NaN,0.0,NaN,2021-07-30
2,0.0,Baseline (Arm 1: C-OPN),MNI0058,26.0,0.0,0.0,6.0,"(1) Unilateral involvement only, usually with ...",25.0,2020-05-22,...,7.0,22.0,24.0,5.0,17.0,59.0,NaN,0.0,NaN,2021-08-18
3,0.0,Baseline (Arm 1: C-OPN),MNI0068,28.0,0.0,11.0,8.0,(2) Bilateral involvement without impairment o...,NaN,2014-01-01,...,0.0,27.0,23.0,5.0,9.0,55.0,NaN,0.0,NaN,2021-08-18
4,0.0,Baseline (Arm 1: C-OPN),MNI0079,22.0,8.0,13.0,10.0,(2) Bilateral involvement without impairment o...,26.0,2017-05-01,...,3.0,17.0,47.0,4.0,22.0,57.0,NaN,0.0,NaN,2022-01-21


### Calculate age

In [13]:
demo_cols = ["participant_id", "dob", "group", "sex"]
demo_var_df[demo_var_df["participant_id"]==participants_with_follow_ups[0]]
baseline_demo_df = demo_var_df[demo_var_df["redcap_event_name"]=="Baseline (Arm 1: C-OPN)"][demo_cols].copy()

index_cols = ["participant_id"] # not using redcap_event_name to allow broadcast of demographics vars
tabular_df = pd.merge(pheno_var_df, baseline_demo_df, on=index_cols, how="left")
tabular_df[tabular_df["participant_id"]==participants_with_follow_ups[0]]

tabular_df.head()

,diagnosis,redcap_event_name,participant_id,Part III: Motor Examination,Part IV: Motor Complications,Part I: Non-Motor Aspects of Experiences of Daily Living (nM-EDL),Part II: Motor Aspects of Experiences of Daily Living (M-EDL),Hoehn and Yahr Stage:,moca_scores,diagnosis_date,...,Digit span backward - longest correct serie (Raw score),Semantic Fluency Actions (Raw score),BNT avec IP,"STROOP GOLDEN : words, self-corrected errors (raw score)",TMT B errors (Raw score),"STROOP GOLDEN : colors, self-corrected errors (raw scores)",neuropsy_date,dob,group,sex
0,0.0,Baseline (Arm 1: C-OPN),MNI0028,29.0,0.0,0.0,2.0,(2) Bilateral involvement without impairment o...,28.0,2020-09-13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1963-07-27,PD (Parkinson's Disease)/Maladie de Parkinson,Male/Masculin
1,NaN,Baseline (Arm 1: C-OPN),MNI0056,58.0,0.0,12.0,22.0,(3) Bilateral disease: mild to moderate disabi...,22.0,2017-09-01,...,3.0,8.0,56.0,NaN,0.0,NaN,2021-07-30,1942-05-21,PD (Parkinson's Disease)/Maladie de Parkinson,Male/Masculin
2,0.0,Baseline (Arm 1: C-OPN),MNI0058,26.0,0.0,0.0,6.0,"(1) Unilateral involvement only, usually with ...",25.0,2020-05-22,...,5.0,17.0,59.0,NaN,0.0,NaN,2021-08-18,1964-03-14,PD (Parkinson's Disease)/Maladie de Parkinson,Male/Masculin
3,0.0,Baseline (Arm 1: C-OPN),MNI0068,28.0,0.0,11.0,8.0,(2) Bilateral involvement without impairment o...,NaN,2014-01-01,...,5.0,9.0,55.0,NaN,0.0,NaN,2021-08-18,1952-05-08,PD (Parkinson's Disease)/Maladie de Parkinson,Male/Masculin
4,0.0,Baseline (Arm 1: C-OPN),MNI0079,22.0,8.0,13.0,10.0,(2) Bilateral involvement without impairment o...,26.0,2017-05-01,...,4.0,22.0,57.0,NaN,0.0,NaN,2022-01-21,1971-11-25,PD (Parkinson's Disease)/Maladie de Parkinson,Female/Féminin


In [14]:
def get_age_at_visit(df, var, dob_col="dob", rounding_digits=2, age_range=(0,100)):
    """ Get age at visit. Expects column name to be: var_date """
    
    age_col = var.split("_")[0]+"_age"
    df[age_col] = df[var] - tabular_df[dob_col]
    df[age_col] = np.round(df[age_col].dt.days / 365.25, rounding_digits)

    if (len(df[df[age_col] > 100]) | len(df[df[age_col] < 0])):
        print(f"Warning: Age values outside range {age_range} for variable {var}")

    return df

In [15]:
age_vars = ["diagnosis_date", "updrs_date", "moca_date", "MRI_date", "neuropsy_date"]

for age_var in age_vars:
    tabular_df = get_age_at_visit(tabular_df, age_var)

tabular_df.head()

,diagnosis,redcap_event_name,participant_id,Part III: Motor Examination,Part IV: Motor Complications,Part I: Non-Motor Aspects of Experiences of Daily Living (nM-EDL),Part II: Motor Aspects of Experiences of Daily Living (M-EDL),Hoehn and Yahr Stage:,moca_scores,diagnosis_date,...,"STROOP GOLDEN : colors, self-corrected errors (raw scores)",neuropsy_date,dob,group,sex,diagnosis_age,updrs_age,moca_age,MRI_age,neuropsy_age
0,0.0,Baseline (Arm 1: C-OPN),MNI0028,29.0,0.0,0.0,2.0,(2) Bilateral involvement without impairment o...,28.0,2020-09-13,...,NaN,NaT,1963-07-27,PD (Parkinson's Disease)/Maladie de Parkinson,Male/Masculin,57.13,60.19,60.19,60.19,NaN
1,NaN,Baseline (Arm 1: C-OPN),MNI0056,58.0,0.0,12.0,22.0,(3) Bilateral disease: mild to moderate disabi...,22.0,2017-09-01,...,NaN,2021-07-30,1942-05-21,PD (Parkinson's Disease)/Maladie de Parkinson,Male/Masculin,75.28,79.06,79.06,79.24,79.19
2,0.0,Baseline (Arm 1: C-OPN),MNI0058,26.0,0.0,0.0,6.0,"(1) Unilateral involvement only, usually with ...",25.0,2020-05-22,...,NaN,2021-08-18,1964-03-14,PD (Parkinson's Disease)/Maladie de Parkinson,Male/Masculin,56.19,57.36,57.36,57.43,57.43
3,0.0,Baseline (Arm 1: C-OPN),MNI0068,28.0,0.0,11.0,8.0,(2) Bilateral involvement without impairment o...,NaN,2014-01-01,...,NaN,2021-08-18,1952-05-08,PD (Parkinson's Disease)/Maladie de Parkinson,Male/Masculin,61.65,69.30,NaN,69.30,69.28
4,0.0,Baseline (Arm 1: C-OPN),MNI0079,22.0,8.0,13.0,10.0,(2) Bilateral involvement without impairment o...,26.0,2017-05-01,...,NaN,2022-01-21,1971-11-25,PD (Parkinson's Disease)/Maladie de Parkinson,Female/Féminin,45.43,50.16,50.08,50.08,50.16


### QPN paper tables

In [104]:
# paper subset filters

redcap_events = ["Baseline (Arm 1: C-OPN)","12 Months Follow-Up/Suivi (Arm 1: C-OPN)","18 Months Follow-Up/Suivi (Arm 1: C-OPN)"]
QPN_groups = {"Healthy control/Contrôle": "control", "PD   (Parkinson's Disease)/Maladie de Parkinson": "PD", np.NaN:"Unknown"}
QPN_sexes = {"Female/Féminin": "Female", "Male/Masculin":"Male"}

n_tabular_participants = tabular_df["participant_id"].nunique()
print(f"Number of participants: {n_tabular_participants}")

paper_df = tabular_df[(tabular_df["redcap_event_name"].isin(redcap_events)) & 
                     (tabular_df["group"].isin(QPN_groups.keys()))].copy()

paper_df["group"] = paper_df["group"].replace(QPN_groups)
paper_df["sex"] = paper_df["sex"].replace(QPN_sexes)

n_paper_participants = paper_df["participant_id"].nunique()
print(f"Number of participants: {n_paper_participants}")
print(f"Number of participants after event and group filter: {n_participants}")

Number of participants: 298
Number of participants: 287
Number of participants after event and group filter: 270


In [106]:
def get_group_table_stats(df, cat_cols, score_cols, groupby_col="redcap_event_name"):
    """ Get table stats for groups. Does not stratify by group! 
    """
    n_cat_cols = len(cat_cols)
    n_score_cols = len(score_cols)
    print(f"Counting {n_cat_cols} and averaging {n_score_cols}")

    table_df = df["redcap_event_name"].value_counts().reset_index()
    print("Starting cat cols")
    for col in cat_cols:
        # print(f"col: {col}")
        cat_count_df = df.groupby([groupby_col])[col].value_counts().unstack().reset_index()
        table_df = pd.merge(table_df, cat_count_df, on=groupby_col, how="left")

    print("Starting score cols")
    for col in score_cols:
        # print(f"col: {col}")
        score_mean_df = df.groupby([groupby_col])[col].mean().round(1)
        score_std_df = df.groupby([groupby_col])[col].std().round(1)
        score_min_df = df.groupby([groupby_col])[col].min().round(1)
        score_max_df = df.groupby([groupby_col])[col].max().round(1)
        score_mean_std_df = score_mean_df.astype(str) + " (" + score_std_df.astype(str) + ")" + " ["  \
        + score_min_df.astype(str) + ", " + score_max_df.astype(str) + "]"
        score_mean_std_df = score_mean_std_df.reset_index()
        table_df = pd.merge(table_df, score_mean_std_df, on=groupby_col, how="left")
    
    return table_df

#### Demo table

In [107]:
demo_vars = ["participant_id", "redcap_event_name", "MRI_age", "sex", "group"]

demo_df = paper_df[demo_vars]
n_participants = demo_df["participant_id"].nunique()
print(f"Number of participants: {n_participants}")
redcap_events = demo_df["redcap_event_name"].unique()
print(f"redcap events: {redcap_events}")
demo_df.head()

Number of participants: 287
redcap events: ['Baseline (Arm 1: C-OPN)' '12 Months Follow-Up/Suivi (Arm 1: C-OPN)'
 '18 Months Follow-Up/Suivi (Arm 1: C-OPN)']


,participant_id,redcap_event_name,MRI_age,sex,group
0,MNI0028,Baseline (Arm 1: C-OPN),60.19,Male,PD
1,MNI0056,Baseline (Arm 1: C-OPN),79.24,Male,PD
2,MNI0058,Baseline (Arm 1: C-OPN),57.43,Male,PD
3,MNI0068,Baseline (Arm 1: C-OPN),69.30,Male,PD
4,MNI0079,Baseline (Arm 1: C-OPN),50.08,Female,PD


In [108]:
# counts
cat_cols = ["sex"]
score_cols = ["MRI_age"]

for dx_group in ["PD", "control"]:
    print(f"*** group: {dx_group} ***")
    dx_df = demo_df[demo_df["group"]==dx_group].copy()
    table_df = get_group_table_stats(dx_df, cat_cols, score_cols)
    print("-"*10)
    print(table_df)
    print("-"*10)


*** group: PD ***
Counting 1 and averaging 1
Starting cat cols
Starting score cols
----------
                          redcap_event_name  count  Female  Male  \
0                   Baseline (Arm 1: C-OPN)    205      65   134   
1  12 Months Follow-Up/Suivi (Arm 1: C-OPN)     19       5    14   
2  18 Months Follow-Up/Suivi (Arm 1: C-OPN)      6       3     3   

                   MRI_age  
0  65.6 (8.9) [40.8, 88.3]  
1  67.2 (9.7) [50.6, 89.3]  
2  65.3 (6.5) [56.9, 73.8]  
----------
*** group: control ***
Counting 1 and averaging 1
Starting cat cols
Starting score cols
----------
                          redcap_event_name  count  Female  Male  \
0                   Baseline (Arm 1: C-OPN)     66    40.0  22.0   
1  12 Months Follow-Up/Suivi (Arm 1: C-OPN)      9     7.0   2.0   
2  18 Months Follow-Up/Suivi (Arm 1: C-OPN)      3     3.0   NaN   

                    MRI_age  
0  62.0 (11.6) [26.0, 82.8]  
1   66.1 (9.9) [46.0, 80.1]  
2   71.2 (9.3) [64.1, 81.8]  
----------


#### Pheno table

In [109]:
index_cols = ["participant_id", "redcap_event_name","group"]
cat_cols = ["diagnosis",'Hoehn and Yahr Stage: ']
score_cols = ['Part I: Non-Motor Aspects of Experiences of Daily Living (nM-EDL)',
              'Part II: Motor Aspects of Experiences of Daily Living (M-EDL)',
              'Part III: Motor Examination', 'Part IV: Motor Complications',               
              'moca_scores']

pheno_vars = index_cols + cat_cols + score_cols
pheno_df = paper_df[pheno_vars]
n_participants = pheno_df["participant_id"].nunique()
print(f"Number of participants: {n_participants}")
redcap_events = demo_df["redcap_event_name"].unique()
print(f"redcap events: {redcap_events}")

pheno_df.head()

Number of participants: 287
redcap events: ['Baseline (Arm 1: C-OPN)' '12 Months Follow-Up/Suivi (Arm 1: C-OPN)'
 '18 Months Follow-Up/Suivi (Arm 1: C-OPN)']


,participant_id,redcap_event_name,group,diagnosis,Hoehn and Yahr Stage:,Part I: Non-Motor Aspects of Experiences of Daily Living (nM-EDL),Part II: Motor Aspects of Experiences of Daily Living (M-EDL),Part III: Motor Examination,Part IV: Motor Complications,moca_scores
0,MNI0028,Baseline (Arm 1: C-OPN),PD,0.0,(2) Bilateral involvement without impairment o...,0.0,2.0,29.0,0.0,28.0
1,MNI0056,Baseline (Arm 1: C-OPN),PD,NaN,(3) Bilateral disease: mild to moderate disabi...,12.0,22.0,58.0,0.0,22.0
2,MNI0058,Baseline (Arm 1: C-OPN),PD,0.0,"(1) Unilateral involvement only, usually with ...",0.0,6.0,26.0,0.0,25.0
3,MNI0068,Baseline (Arm 1: C-OPN),PD,0.0,(2) Bilateral involvement without impairment o...,11.0,8.0,28.0,0.0,NaN
4,MNI0079,Baseline (Arm 1: C-OPN),PD,0.0,(2) Bilateral involvement without impairment o...,13.0,10.0,22.0,8.0,26.0


In [110]:
for dx_group in ["PD", "control"]:
    print(f"*** group: {dx_group} ***")
    dx_df = pheno_df[pheno_df["group"]==dx_group].copy()
    table_df = get_group_table_stats(dx_df, cat_cols, score_cols)
    print("-"*10)
    print(table_df)
    print("-"*10)


*** group: PD ***
Counting 2 and averaging 5
Starting cat cols
Starting score cols
----------
                          redcap_event_name  count    0.0  (0) Asymptomatic  \
0                   Baseline (Arm 1: C-OPN)    205  164.0               4.0   
1  12 Months Follow-Up/Suivi (Arm 1: C-OPN)     19    NaN               NaN   
2  18 Months Follow-Up/Suivi (Arm 1: C-OPN)      6    NaN               NaN   

   (1) Unilateral involvement only, usually with minimal or no functional disability  \
0                                               35.0                                   
1                                                NaN                                   
2                                                NaN                                   

   (2) Bilateral involvement without impairment of balance  \
0                                              100.0         
1                                               10.0         
2                                                2

### Neuropsy scores

In [111]:
select_cols = neuropsych_df.isna().sum().sort_values(ascending=True).index[:20]

neuropsych_df[select_cols]

,redcap_event_name,Letter fluency TOTAL [automatique],HVLT Trial 2 (Raw score),HVLT TOTAL [automatique],participant_id,visit,neuropsy_date,HVLT Trial 1 (Raw score),HVLT Trial 3 (Raw score),"HVLT Trial total 1,2,3 (Raw score)","TimePoint (based on REDCap; baseline, 18m, 36m, etc.)",HVLT Trial 4 delayed (Raw score),TMT A time seconds (Raw score),Clock Command (Raw score),Semantic Fluency Total (Raw score),Semantic fluency TOTAL [automatique],Clock Copy (Raw score),Letter Fluency S (Raw score),Semantic Fluency Actions (Raw score),Letter Fluency Total (Raw score)
0,Baseline (Arm 1: C-OPN),55,5.0,17.0,PD00119,V01,2017-12-07,3.0,9.0,17.0,baseline,6.0,47.0,7.0,50.0,50.0,10.0,16.0,25.0,55.0
1,Baseline (Arm 1: C-OPN),35,6.0,19.0,PD01199,V01,2018-03-12,5.0,8.0,19.0,baseline,7.0,56.0,10.0,34.0,34.0,9.0,15.0,17.0,35.0
2,Baseline (Arm 1: C-OPN),22,7.0,21.0,PD01224,V01,2018-03-12,5.0,9.0,21.0,baseline,8.0,53.0,7.0,36.0,36.0,7.0,6.0,15.0,22.0
3,Baseline (Arm 1: C-OPN),32,7.0,19.0,PD01160,V01,2018-08-06,5.0,7.0,19.0,baseline,4.0,20.0,NaN,36.0,35.0,NaN,9.0,18.0,32.0
4,Baseline (Arm 1: C-OPN),54,9.0,29.0,PD00869,V01,2018-07-31,8.0,12.0,29.0,baseline,12.0,44.0,9.0,44.0,44.0,10.0,21.0,22.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,nan,49,10.0,29.0,PD01563,V02,2023-03-05,7.0,12.0,29.0,follow-up,12.0,23.0,8.0,42.0,42.0,10.0,21.0,18.0,49.0
202,18 Months Follow-Up/Suivi (Arm 1: C-OPN),36,7.0,21.0,PD01591,V02,2023-05-05,6.0,8.0,21.0,follow-up,6.0,24.0,10.0,28.0,28.0,10.0,12.0,13.0,36.0
203,42 Months Follow-Up/Suivi (Arm 1: C-OPN),28,7.0,20.0,PD01022,V03,2023-10-05,4.0,9.0,20.0,follow-up,6.0,28.0,10.0,34.0,34.0,10.0,9.0,17.0,28.0
204,nan,27,6.0,20.0,MNI0280,V02,2023-05-19,6.0,8.0,20.0,follow-up,3.0,58.0,8.0,27.0,27.0,7.0,6.0,12.0,27.0


In [112]:
index_cols = ["participant_id", "redcap_event_name","group"]
cat_cols = []
score_cols = [
                "HVLT Trial total 1,2,3 (Raw score)",
                "Clock Command (Raw score)",
                "Clock Copy (Raw score)",
                "Semantic Fluency Total (Raw score)",
                "RCFT Copy (Raw score)",
                "Letter Fluency Total (Raw score)",
                "STROOP GOLDEN : words, self-corrected errors (raw score)",
                "STROOP GOLDEN : colors, self-corrected errors (raw scores)",
                "Stroop GOLDEN : ink, self-corrected errors (raw score)",                
                ]

neuropsy_vars = index_cols + cat_cols + score_cols
neuropsy_df = paper_df[neuropsy_vars]
n_participants = neuropsy_df["participant_id"].nunique()
print(f"Number of participants: {n_participants}")

neuropsy_df.head()

Number of participants: 287


,participant_id,redcap_event_name,group,"HVLT Trial total 1,2,3 (Raw score)",Clock Command (Raw score),Clock Copy (Raw score),Semantic Fluency Total (Raw score),RCFT Copy (Raw score),Letter Fluency Total (Raw score),"STROOP GOLDEN : words, self-corrected errors (raw score)","STROOP GOLDEN : colors, self-corrected errors (raw scores)","Stroop GOLDEN : ink, self-corrected errors (raw score)"
0,MNI0028,Baseline (Arm 1: C-OPN),PD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MNI0056,Baseline (Arm 1: C-OPN),PD,11.0,6.0,9.0,20.0,28.0,17.0,NaN,NaN,NaN
2,MNI0058,Baseline (Arm 1: C-OPN),PD,26.0,10.0,9.0,37.0,26.0,24.0,NaN,NaN,NaN
3,MNI0068,Baseline (Arm 1: C-OPN),PD,21.0,8.0,10.0,27.0,21.5,23.0,NaN,NaN,NaN
4,MNI0079,Baseline (Arm 1: C-OPN),PD,31.0,9.0,10.0,46.0,30.0,47.0,NaN,NaN,NaN


In [113]:
for dx_group in ["PD", "control"]:
    print(f"*** group: {dx_group} ***")
    dx_df = neuropsy_df[neuropsy_df["group"]==dx_group].copy()
    table_df = get_group_table_stats(dx_df, cat_cols, score_cols)
    print("-"*10)
    print(table_df)
    print("-"*10)


*** group: PD ***
Counting 0 and averaging 9
Starting cat cols
Starting score cols
----------
                          redcap_event_name  count  \
0                   Baseline (Arm 1: C-OPN)    205   
1  12 Months Follow-Up/Suivi (Arm 1: C-OPN)     19   
2  18 Months Follow-Up/Suivi (Arm 1: C-OPN)      6   

  HVLT Trial total 1,2,3 (Raw score) Clock Command (Raw score)  \
0             21.3 (5.6) [7.0, 34.0]     8.0 (1.9) [0.0, 10.0]   
1            26.9 (3.9) [23.0, 33.0]     9.2 (0.8) [8.0, 10.0]   
2            24.5 (4.9) [21.0, 28.0]     8.5 (2.1) [7.0, 10.0]   

  Clock Copy (Raw score) Semantic Fluency Total (Raw score)  \
0  8.8 (1.2) [3.0, 10.0]            34.0 (9.8) [12.0, 61.0]   
1  9.7 (0.5) [9.0, 10.0]            36.6 (7.7) [25.0, 51.0]   
2  9.5 (0.7) [9.0, 10.0]           37.5 (13.4) [28.0, 47.0]   

     RCFT Copy (Raw score) Letter Fluency Total (Raw score)  \
0   26.6 (6.3) [4.0, 36.0]         36.1 (12.6) [12.0, 76.0]   
1  31.7 (2.1) [29.0, 34.0]         44.8 (16.3

In [156]:
index_cols = ["participant_id","redcap_event_name"]
event_cols = ['Baseline (Arm 1: C-OPN)', '12 Months Follow-Up/Suivi (Arm 1: C-OPN)', '18 Months Follow-Up/Suivi (Arm 1: C-OPN)']

print_col_dict = {
    "demo_cols" : ["dob","sex","group"],
    "dx_cols" : ["diagnosis", "diagnosis_date", 'Hoehn and Yahr Stage: '],
    "updrs_cols" : ['Part I: Non-Motor Aspects of Experiences of Daily Living (nM-EDL)',
                'Part II: Motor Aspects of Experiences of Daily Living (M-EDL)',
                'Part III: Motor Examination', 'Part IV: Motor Complications',"updrs_date"],
    "moca_cols" : ["moca_scores"]
}

for event_name in event_cols:
    print("-"*50)
    print(f"event: {event_name}")
    for var_set, pcl in print_col_dict.items():
        tabular_subset_df = tabular_df[tabular_df["redcap_event_name"]==event_name][index_cols + pcl]
        bagel_subset_df = tabular_subset_df.copy()
        bagel_subset_df[pcl] = tabular_subset_df[pcl].isna()
        missing_count = dict(bagel_subset_df[pcl].sum(axis=0))
        global_missing_count = bagel_subset_df[pcl].sum(axis=0).sum()
        print(f"{var_set}\n: {missing_count}")
        bagel_subset_df[pcl] = bagel_subset_df[pcl].replace({True:"MISSING", False:"AVAILABLE"})
        
        if global_missing_count > 0:
            event_name_clean = event_name.replace(" ","_").replace("/", "_")
            save_file = f"{tabular_data_release_dir}/tabular/print_lists/bagel_{event_name_clean}_{var_set}.csv"
            print(f"saving at: {save_file}")
            bagel_subset_df.to_csv(save_file, index=False)
        else:
            print(f"No missing data for {var_set}")

--------------------------------------------------
event: Baseline (Arm 1: C-OPN)
demo_cols
: {'dob': 13, 'sex': 25, 'group': 17}
saving at: /home/nikhil/projects/Parkinsons/qpn//releases//Jan_2024//tabular/print_lists/bagel_Baseline_(Arm_1:_C-OPN)_demo_cols.csv
dx_cols
: {'diagnosis': 128, 'diagnosis_date': 136, 'Hoehn and Yahr Stage: ': 145}
saving at: /home/nikhil/projects/Parkinsons/qpn//releases//Jan_2024//tabular/print_lists/bagel_Baseline_(Arm_1:_C-OPN)_dx_cols.csv
updrs_cols
: {'Part I: Non-Motor Aspects of Experiences of Daily Living (nM-EDL)': 146, 'Part II: Motor Aspects of Experiences of Daily Living (M-EDL)': 147, 'Part III: Motor Examination': 144, 'Part IV: Motor Complications': 147, 'updrs_date': 144}
saving at: /home/nikhil/projects/Parkinsons/qpn//releases//Jan_2024//tabular/print_lists/bagel_Baseline_(Arm_1:_C-OPN)_updrs_cols.csv
moca_cols
: {'moca_scores': 105}
saving at: /home/nikhil/projects/Parkinsons/qpn//releases//Jan_2024//tabular/print_lists/bagel_Baseline_(A

In [129]:
tabular_data_release_dir

'/home/nikhil/projects/Parkinsons/qpn//releases//Jan_2024/'